## Library

In [2]:
import pandas as pd 

## Data

In [3]:
data = pd.read_csv("data/nigeria_houses_data.csv")

In [4]:
# Change state name error (Anambara <=> Anambra)
data.loc[data["state"] == "Anambara", "state"] = "Anambra"

In [5]:
for state in list(data["state"].unique()):
    print(state)
    print(list(data.loc[data["state"] == state]["town"].unique()))
    print("\n")

Abuja
['Mabushi', 'Katampe', 'Lokogoma District', 'Kaura', 'Galadimawa', 'Gwarinpa', 'Lugbe District', 'Jahi', 'Orozo', 'Guzape District', 'Idu Industrial', 'Utako', 'Kuje', 'Life Camp', 'Dape', 'Gaduwa', 'Dakwo', 'Asokoro District', 'Wuye', 'Kubwa', 'Apo', 'Wuse 2', 'Durumi', 'Maitama District', 'Karsana', 'Wuse', 'Kurudu', 'Karmo', 'Gudu', 'Kukwaba', 'Mbora (Nbora)', 'Jabi', 'Garki', 'Karshi', 'Kado', 'Nyanya', 'Kyami', 'Dutse', 'Karu', 'Kafe', 'Dakibiyu', 'Bwari', 'Kagini', 'Mpape', 'Gwagwalada', 'Diplomatic Zones', 'Kabusa', 'Dei-Dei', 'Duboyi', 'Jikwoyi', 'Central Business District', 'Wumba', 'Mararaba']


Lagos
['Lekki', 'Ajah', 'Epe', 'Victoria Island (VI)', 'Ikeja', 'Ikoyi', 'Magodo', 'Ibeju Lekki', 'Yaba', 'Ifako-Ijaiye', 'Agege', 'Ikorodu', 'Isheri North', 'Isheri', 'Ipaja', 'Mushin', 'Ejigbo', 'Isolo', 'Ojodu', 'Alimosho', 'Shomolu', 'Ogudu', 'Surulere', 'Ayobo', 'Ikotun', 'Maryland', 'Gbagada', 'Idimu', 'Ojo', 'Kosofe', 'Ilupeju', 'Ketu', 'Ojota', 'Oshodi', 'Amuwo Odofin', 

## Append Geo-political Zone

In [49]:
geo_zones = [
    "North Central", "South West", "South South", "South West", "South West", "South East", "South East", "South South",
    "South East", "North West", "North Central", "North Central", "South East", "South South", "South South", "South West",
    "South West", "South South", "North Central", "North Central", "North West", "North West", "South South", "North East", 
    "North Central"
]

state_zone = {state: zone for state, zone in zip(list(data["state"].unique()), geo_zones)}

In [50]:
data["geo_zone"] = data["state"].map(state_zone)

data[["geo_zone", "state"]]

,geo_zone,state
0,North Central,Abuja
1,North Central,Abuja
2,South West,Lagos
3,South West,Lagos
4,South West,Lagos
...,...,...
24321,North Central,Abuja
24322,South West,Ogun
24323,South West,Lagos
24324,South West,Lagos


## Append State GDP

In [25]:
import requests
from bs4 import BeautifulSoup

In [26]:
# Replace 'URL' with the actual Wikipedia page URL
url = 'https://en.wikipedia.org/wiki/List_of_Nigerian_states_by_GDP'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Select table class you want to scrape
table = soup.find('table', class_='wikitable')

In [27]:
table

<table class="wikitable sortable plainrowheaders">
<tbody><tr>
<th align="left" scope="col">Rank
</th>
<th align="left" scope="col">State
</th>
<th scope="col">GDP <small><br/>(tril. ₦)</small>
</th>
<th scope="col">GDP <small><br/>(bil. US$)</small>
</th>
<th scope="col">GDP PPP<small><br/>(bil. int$)</small>
</th>
<th scope="col">GDP per capita<small><br/>(US$)</small>
</th>
<th scope="col">GDP per capita PPP<small><br/>(int$)</small>
</th></tr>
<tr>
<td align="left">1
</td>
<td align="left"><a href="/wiki/Lagos_State" title="Lagos State">Lagos State</a>
</td>
<td align="center">₦41.17
</td>
<td align="center">102.01
</td>
<td align="center">266.55
</td>
<td align="center">6,614
</td>
<td align="center">17,282
</td></tr>
<tr>
<td align="left">2
</td>
<td align="left"><a href="/wiki/Rivers_State" title="Rivers State">Rivers State</a>
</td>
<td align="center">₦7.96
</td>
<td align="center">19.72
</td>
<td align="center">51.52
</td>
<td align="center">2,277
</td>
<td align="center">5,94

In [ ]:
df_list = pd.read_html(str(table))
# Assuming there's only one table, select the first element from the list
df = df_list[0]

In [29]:
gdp_data = []
# Iterate through table rows
for row in table.find_all('tr'):  
    row_data = []
    # Iterate through cells (tds and ths)
    for cell in row.find_all(['td', 'th']):  
        # Extract text with stripping whitespaces
        cell_text = cell.get_text(strip=True)  
        row_data.append(cell_text)
    gdp_data.append(row_data)

# Create DataFrame from extracted data
df = pd.DataFrame(gdp_data)  

In [48]:
# Rename Columns
gdp_df.columns = [
    "rank", "state", "GDP_tril_naira", "GDP_bill_dollars", "GDP_PPP_bill_dollars", "GDP_per_Capital_dollars", 
    "GDP_per_capital_PPP_dollars"
]

gdp_df = gdp_df.iloc[1:, :]

In [49]:
gdp_df.head()

,rank,State,GDP_tril_naira,GDP_bill_dollars,GDP_PPP_bill_dollars,GDP_per_Capital_dollars,GDP_per_capital_PPP_dollars
1,1,Lagos State,₦41.17,102.01,266.55,"6,614","17,282"
2,2,Rivers State,₦7.96,19.72,51.52,"2,277","5,949"
3,3,Akwa Ibom State,₦7.77,19.25,50.30,"2,962","7,739"
4,4,Imo State,₦7.68,19.02,49.69,"2,996","7,828"
5,5,Delta State,₦6.19,15.33,40.05,"2,306","6,025"


In [50]:
# Clean GDP_tril_naira column by replacing "₦"
gdp_df["GDP_tril_naira"] = gdp_df["GDP_tril_naira"].str.replace("₦", "")

gdp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 1 to 36
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   rank                         36 non-null     object
 1   State                        36 non-null     object
 2   GDP_tril_naira               36 non-null     object
 3   GDP_bill_dollars             36 non-null     object
 4   GDP_PPP_bill_dollars         36 non-null     object
 5   GDP_per_Capital_dollars      36 non-null     object
 6   GDP_per_capital_PPP_dollars  36 non-null     object
dtypes: object(7)
memory usage: 2.1+ KB


In [51]:
# Remove all "," and convert column to float
flo_cols = ["GDP_tril_naira", "GDP_bill_dollars", "GDP_PPP_bill_dollars", "GDP_per_Capital_dollars", "GDP_per_capital_PPP_dollars"]
for col in flo_cols:
    gdp_df[col] = gdp_df[col].str.replace(",", "")
    gdp_df[col] = gdp_df[col].astype("float64")

In [52]:
gdp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 1 to 36
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   rank                         36 non-null     object 
 1   State                        36 non-null     object 
 2   GDP_tril_naira               36 non-null     float64
 3   GDP_bill_dollars             36 non-null     float64
 4   GDP_PPP_bill_dollars         36 non-null     float64
 5   GDP_per_Capital_dollars      36 non-null     float64
 6   GDP_per_capital_PPP_dollars  36 non-null     float64
dtypes: float64(5), object(2)
memory usage: 2.1+ KB


In [53]:
gdp_df.head()

,rank,State,GDP_tril_naira,GDP_bill_dollars,GDP_PPP_bill_dollars,GDP_per_Capital_dollars,GDP_per_capital_PPP_dollars
1,1,Lagos State,41.17,102.01,266.55,6614.0,17282.0
2,2,Rivers State,7.96,19.72,51.52,2277.0,5949.0
3,3,Akwa Ibom State,7.77,19.25,50.30,2962.0,7739.0
4,4,Imo State,7.68,19.02,49.69,2996.0,7828.0
5,5,Delta State,6.19,15.33,40.05,2306.0,6025.0


In [58]:
# Remove the State word from the state variable
gdp_df["state"] = gdp_df["state"].str.replace("State", "").str.strip()

In [36]:
# Create the missing abuja records
abuja_estimate = pd.DataFrame({
    "rank": 0,
    "state": "Abuja",
    "GDP_tril_naira": 0,
    "GDP_bill_dollars": 5.4,
    "GDP_PPP_bill_dollars": 0,
    "GDP_per_Capital_dollars": 1292,
    "GDP_per_capital_PPP_dollars": 0
}, index=[0])


gdp_df = pd.concat([gdp_df, abuja_estimate], axis=0)

In [37]:
# Save gdp_df as a csv file in the data directory
gdp_df.to_csv("data/est_GDP_2021.csv", index=False)

In [52]:
# Save the combined dataset 
gdp_df = pd.read_csv("data/est_GDP_2021.csv")
gdp_df["state"].unique()

array(['Lagos', 'Rivers', 'Akwa Ibom', 'Imo', 'Delta', 'Anambra', 'Ondo',
       'Ogun', 'Bayelsa', 'Niger', 'Kaduna', 'Benue', 'Kano',
       'Cross River', 'Edo', 'Kogi', 'Oyo', 'Abia', 'Katsina', 'Sokoto',
       'Adamawa', 'Bauchi', 'Ekiti', 'Osun', 'Ebonyi', 'Jigawa', 'Gombe',
       'Taraba', 'Borno', 'Nasarawa', 'Kebbi', 'Zamfara', 'Plateau',
       'Enugu', 'Kwara', 'Yobe', 'Abuja'], dtype=object)

In [55]:
# Combine the property dataset and the gdp_df together.
data = data.merge(gdp_df[["state", "GDP_per_Capital_dollars"]], on="state", how="left")

data.head()

In [56]:
# Check for null values in the data
data.isnull().sum()

bedrooms                   0
bathrooms                  0
toilets                    0
parking_space              0
title                      0
town                       0
state                      0
price                      0
geo_zone                   0
GDP_per_Capital_dollars    0
dtype: int64

In [86]:
# Save the combined dataset in the data directory
data.to_csv("data/nig_housing_price.csv", index=False)

## Append Historical Crime Rate (2017)

In [3]:
# Import the downloded Crime statistics summary
crime_df = pd.read_excel("data/CRIME STATISTICS 2017.xlsx", sheet_name="Summary")

In [8]:
# Rename columns
crime_df.columns = [
    "state", "OFFENCES AGAINST PERSONS", "OFFENCES AGAINST PROPERTY", "OFFENCES AGAINST LAWFUL AUTHORITY", "total_crime_2017"
]

# Drop the first row containing missing or bad records
crime_df = crime_df.iloc[1:, :]
crime_df.head()

,state,OFFENCES AGAINST PERSONS,OFFENCES AGAINST PROPERTY,OFFENCES AGAINST LAWFUL AUTHORITY,total_crime_2017
1,FCT-ABUJA,1183,4160,39,5382
2,AKWA-IBOM,702,593,378,1673
3,ANAMBRA,127,1030,731,1888
4,ABIA,9816,2320,272,12408
5,ADAMAWA,811,1406,52,2269


In [17]:
# Read in the latest cleaned property data. 
data = pd.read_csv("data/nig_housing_price.csv")
data

,bedrooms,bathrooms,toilets,parking_space,title,town,state,price,geo_zone,GDP_per_Capital_dollars
0,6.0,5.0,5.0,4.0,Detached Duplex,Mabushi,Abuja,450000000.0,North Central,1292.0
1,4.0,5.0,5.0,4.0,Terraced Duplexes,Katampe,Abuja,800000000.0,North Central,1292.0
2,4.0,5.0,5.0,4.0,Detached Duplex,Lekki,Lagos,120000000.0,South West,6614.0
3,4.0,4.0,5.0,6.0,Detached Duplex,Ajah,Lagos,40000000.0,South West,6614.0
4,4.0,4.0,5.0,2.0,Semi Detached Duplex,Lekki,Lagos,75000000.0,South West,6614.0
...,...,...,...,...,...,...,...,...,...,...
24321,2.0,2.0,2.0,4.0,Block of Flats,Kabusa,Abuja,15000000.0,North Central,1292.0
24322,4.0,5.0,5.0,4.0,Block of Flats,Ado-Odo/Ota,Ogun,25000000.0,South West,2024.0
24323,4.0,5.0,5.0,4.0,Detached Duplex,Lekki,Lagos,68000000.0,South West,6614.0
24324,3.0,4.0,4.0,3.0,Block of Flats,Victoria Island (VI),Lagos,78000000.0,South West,6614.0


In [18]:
# Check for any spelling errors in the state column
data["state"].unique()

array(['Abuja', 'Lagos', 'Edo', 'Ogun', 'Oyo', 'Imo', 'Anambara',
       'Rivers', 'Enugu', 'Kaduna', 'Kwara', 'Nasarawa', 'Abia', 'Delta',
       'Akwa Ibom', 'Osun', 'Ekiti', 'Cross River', 'Kogi', 'Plateau',
       'Kano', 'Katsina', 'Bayelsa', 'Borno', 'Niger'], dtype=object)

In [ ]:
# Transform the state string to title
crime_df["state"] = crime_df["state"].str.title()

In [23]:
# Check for any difference in the various state values
crime_df["state"].unique()

array(['Abuja', 'Akwa Ibom', 'Anambra', 'Abia', 'Adamawa', 'Bauchi',
       'Benue', 'Borno', 'Bayelsa', 'C/River', 'Delta', 'Ebonyi', 'Edo',
       'Enugu', 'Gombe', 'Ekiti', 'Imo', 'Jigawa', 'Kaduna', 'Kano',
       'Katsina', 'Kebbi', 'Kogi', 'Kwara', 'Lagos', 'Niger', 'Nasarawa',
       'Ogun', 'Ondo', 'Oyo', 'Osun', 'Plateau', 'Rivers', 'Sokoto',
       'Taraba', 'Yobe', 'Zamfara', 'Railway Command',
       'Ports Authority Command (Pap)', 'Total'], dtype=object)

In [19]:
# Change state values spelling difference
crime_df.loc[crime_df["state"] == "Fct-Abuja", "state"] = "Abuja"
crime_df.loc[crime_df["state"] == "Akwa-Ibom", "state"] = "Akwa Ibom"
crime_df.loc[crime_df["state"] == "C/River", "state"] = "Cross River"

In [66]:
# Save the cleaned crime summary data in the data directory
crime_df.to_csv("data/clean_crime_2017.csv", index=False)

In [67]:
# Combine the property dataset (including the GDP per capital) and the Crime summary together
data = data.merge(crime_df[["state", "total_crime_2017"]], how="left", on="state")

In [68]:
# Save the property dataset in the data directory
data.to_csv("data/nig_housing_price.csv", index=False)